In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/document.txt")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
)

def load_memory(_):
    return memory.load_memory_variables({})['history']

#vectorstore.similarity_search("where does winston live")

prompt = ChatPromptTemplate.from_messages([
    (
        'system', 
        """
        You are a helpful assistant.
        Answer questions using only the following context.
        If you don't know the answer just say you don't know, don't make it up:\n\n{context}""",
    ),
    MessagesPlaceholder(variable_name='history'), 
    ('human', '{question}')
])

chain = {'context': retriever, 'question': RunnablePassthrough(), 'history' : RunnableLambda(load_memory)} | prompt | llm

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {'input': question},
        {'output': result},
    )
    print(result)
    
invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia?")

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)